# Aftermarket Toolkit: Notification System

This notebook documents the comprehensive notification system implementation for the Aftermarket Toolkit application.

## System Requirements

The notification system should satisfy the following requirements:

1. Track and display different notification types (friend requests, messages, forum responses, etc.)
2. Mark notifications as read/unread
3. Provide real-time notification updates using AJAX
4. Allow users to configure notification preferences
5. Support both in-app and email notifications
6. Optimize database queries for performance with large notification volumes

## Database Schema

The notification system uses the following database structure:

In [ ]:
-- Notifications Table
CREATE TABLE `notifications` (
  `id` int(11) NOT NULL AUTO_INCREMENT,
  `user_id` int(11) NOT NULL,
  `type` varchar(50) NOT NULL,
  `related_id` int(11) DEFAULT NULL,
  `from_user_id` int(11) DEFAULT NULL,
  `content` text DEFAULT NULL,
  `is_read` tinyint(1) NOT NULL DEFAULT 0,
  `created_at` timestamp NOT NULL DEFAULT current_timestamp(),
  PRIMARY KEY (`id`),
  KEY `user_id` (`user_id`),
  KEY `from_user_id` (`from_user_id`),
  KEY `type_index` (`type`),
  KEY `is_read_index` (`is_read`),
  KEY `created_at_index` (`created_at`)
);

-- User Settings Table
CREATE TABLE `user_settings` (
  `id` int(11) NOT NULL AUTO_INCREMENT,
  `user_id` int(11) NOT NULL,
  `settings` text NOT NULL,
  `updated_at` timestamp NOT NULL DEFAULT current_timestamp() ON UPDATE current_timestamp(),
  PRIMARY KEY (`id`),
  UNIQUE KEY `user_id` (`user_id`)
);

## Notification Types

The system supports the following notification types:

1. `friend_request`: When a user sends a friend request
2. `message`: When a user receives a private message
3. `forum_response`: When someone replies to a user's forum thread
4. `listing_comment`: When someone comments on a user's listing
5. `system`: System notifications (announcements, updates, etc.)

## Core Notification Functions

The notification system includes the following core functions defined in `notification_handler.php`:

In [ ]:
/**
 * Create a notification
 * 
 * @param object $conn Database connection
 * @param int $userId User ID to notify
 * @param string $type Type of notification (friend_request, message, forum_response)
 * @param int $relatedId ID of the related item (request, message, or response)
 * @param string $content Description of the notification
 * @return bool Success or failure
 */
function createNotification($conn, $userId, $type, $relatedId, $fromUserId = null, $content = '') {
    // Check for duplicates and create notification
}

/**
 * Mark notifications as read
 */
function markNotificationAsRead($conn, $notificationId, $userId) {
    // Mark a notification as read
}

/**
 * Get user notifications
 */
function getUserNotifications($conn, $userId, $limit = 10, $unreadOnly = false) {
    // Retrieve user notifications
}

## Notification Display

Notifications are displayed in a dropdown menu in the site header, with real-time updates via AJAX. The notification badge shows the count of unread notifications.

In [ ]:
<!-- Notification Dropdown Example -->
<div class="notification-dropdown">
    <div class="notification-header">
        <h3>Notifications</h3>
        <button class="mark-all-read">Mark all as read</button>
    </div>
    
    <div class="notification-list">
        <!-- Individual notification items generated by renderNotificationsDropdown() -->
        <div class="notification-item unread" data-id="123">
            <div class="notification-icon"><i class="fas fa-user-plus"></i></div>
            <div class="notification-content">
                <div class="notification-text"><strong>User123</strong> sent you a friend request</div>
                <div class="notification-time">2 hours ago</div>
            </div>
            <div class="notification-mark-read"><i class="fas fa-check"></i></div>
            <a href="/aftermarket_toolkit/public/friends.php" class="notification-link"></a>
        </div>
    </div>
</div>

## AJAX Implementation

The notification system uses AJAX to provide real-time updates without page refresh:

In [ ]:
// Fetch notifications via AJAX
function fetchNotifications() {
    $.ajax({
        url: '/aftermarket_toolkit/public/api/notifications.php',
        type: 'GET',
        dataType: 'json',
        success: function(response) {
            if (response.success) {
                updateNotificationBadge(response.counts.total);
                updateNotificationDropdown(response.notifications);
            }
        }
    });
}

// Mark notification as read
function markNotificationAsRead(notificationId, event) {
    event.stopPropagation();
    
    $.ajax({
        url: '/aftermarket_toolkit/public/api/notifications.php',
        type: 'POST',
        data: {
            action: 'mark_read',
            notification_id: notificationId
        },
        dataType: 'json',
        success: function(response) {
            if (response.success) {
                // Update UI
                $(`[data-notification-id="${notificationId}"]`).removeClass('unread');
                updateNotificationBadge(response.counts.total);
            }
        }
    });
}

// Update the notification badge count
function updateNotificationBadge(count) {
    const $badge = $('#notification-badge');
    
    if (count > 0) {
        $badge.text(count).show();
    } else {
        $badge.text('').hide();
    }
}

// Poll for new notifications every 60 seconds
setInterval(fetchNotifications, 60000);

## API Endpoints

The notification system provides the following API endpoints:

### GET /api/notifications.php

Returns the user's notifications and counts.

**Response:**
```json
{
  "success": true,
  "notifications": [...],
  "counts": {
    "friend_request": 1,
    "message": 2,
    "forum_response": 0,
    "listing_comment": 0,
    "total": 3
  }
}
```

### POST /api/notifications.php

**Actions:**

- `mark_read`: Mark a notification as read
- `mark_all_read`: Mark all notifications as read
- `delete`: Delete a notification

**Parameters:**
- `action`: The action to perform
- `notification_id`: For 'mark_read' and 'delete' actions
- `type`: (Optional) For 'mark_all_read' to mark only notifications of a specific type

## User Notification Settings

Users can configure their notification preferences in their profile settings. These settings control both in-app and email notifications.

In [ ]:
<!-- Notification Settings Form Example -->
<form id="notification-settings-form" method="post">
    <h3>In-app Notifications</h3>
    
    <div class="form-group">
        <input type="checkbox" id="notify_friend_requests" name="notify_friend_requests" value="1" checked>
        <label for="notify_friend_requests">Friend Requests</label>
    </div>
    
    <div class="form-group">
        <input type="checkbox" id="notify_messages" name="notify_messages" value="1" checked>
        <label for="notify_messages">New Messages</label>
    </div>
    
    <div class="form-group">
        <input type="checkbox" id="notify_forum_responses" name="notify_forum_responses" value="1" checked>
        <label for="notify_forum_responses">Forum Responses</label>
    </div>
    
    <div class="form-group">
        <input type="checkbox" id="notify_listing_comments" name="notify_listing_comments" value="1" checked>
        <label for="notify_listing_comments">Listing Comments</label>
    </div>
    
    <h3>Email Notifications</h3>
    
    <div class="form-group">
        <input type="checkbox" id="email_friend_requests" name="email_friend_requests" value="1" checked>
        <label for="email_friend_requests">Friend Requests</label>
    </div>
    
    <div class="form-group">
        <input type="checkbox" id="email_messages" name="email_messages" value="1" checked>
        <label for="email_messages">New Messages</label>
    </div>
    
    <div class="form-group">
        <input type="checkbox" id="email_forum_responses" name="email_forum_responses" value="1">
        <label for="email_forum_responses">Forum Responses</label>
    </div>
    
    <div class="form-group">
        <input type="checkbox" id="email_listing_comments" name="email_listing_comments" value="1">
        <label for="email_listing_comments">Listing Comments</label>
    </div>
    
    <button type="submit" class="btn btn-primary">Save Settings</button>
</form>

## Email Notifications

If enabled in user settings, the system will send email notifications for various events:

In [ ]:
/**
 * Send notification email
 *
 * @param object $conn Database connection
 * @param int $userId User ID
 * @param array $notification Notification data
 * @return bool Success status
 */
function sendNotificationEmail($conn, $userId, $notification) {
    // Get user email and settings
    $userQuery = "SELECT email FROM users WHERE id = ? LIMIT 1";
    $userStmt = $conn->prepare($userQuery);
    $userStmt->bind_param("i", $userId);
    $userStmt->execute();
    $userResult = $userStmt->get_result();
    
    if (!($user = $userResult->fetch_assoc())) {
        return false;
    }
    
    $settings = getNotificationSettings($conn, $userId);
    $emailSetting = 'email_' . $notification['type'];
    
    // Check if email notifications are enabled for this type
    if (!isset($settings[$emailSetting]) || !$settings[$emailSetting]) {
        return false;
    }
    
    // Prepare email content
    $subject = 'New Notification: ' . ucfirst(str_replace('_', ' ', $notification['type']));
    $content = formatNotification($notification);
    
    // Send email
    return sendEmail($user['email'], $subject, $content);
}

## Performance Considerations

1. **Database Indexing**: The notification tables are indexed on frequently queried columns (`user_id`, `is_read`, `created_at`, etc.) to improve query performance.

2. **Pagination**: The system implements pagination to limit the number of notifications fetched at once.

3. **Caching**: Notification counts are cached to reduce database load for frequently accessed information.

4. **Bulk Operations**: The system provides methods for bulk operations (mark all as read, delete all) to reduce the number of database queries.

5. **Optimized Queries**: Joins are minimized and queries are structured to retrieve all necessary data in a single operation.

## Implementation Roadmap

1. **Phase 1** ✅
   - Database schema setup
   - Basic notification creation and display
   - Core notification handler functions

2. **Phase 2** 🔄
   - AJAX real-time notification updates
   - User notification settings
   - Email notifications

3. **Phase 3** 📅
   - Push notifications via service workers
   - Notification analytics
   - Performance optimizations for high-volume scenarios
   - Mobile app notifications

## Conclusion

The notification system provides a comprehensive solution for tracking and displaying user notifications across the Aftermarket Toolkit application. It supports multiple notification types, user configuration, and is designed to scale with the application's growth.